In [14]:
import numpy as np
from scipy.io import loadmat

data_3factors = loadmat('Data_3factors.mat')
data_active = loadmat('Data_active.mat')

factors = np.column_stack([data_3factors['MKTRF'].flatten(), data_3factors['SMB'].flatten(), data_3factors['HML'].flatten()]) / 100

active_data = np.column_stack([data_active['BEN'].flatten(), data_active['BLK'].flatten(), data_active['BWG'].flatten(), data_active['BX'].flatten(), data_active['FHI'].flatten(), data_active['SEIC'].flatten(), data_active['VGT'].flatten()])

returns_SPX = np.diff(np.log(data_3factors['SPX'].flatten()))
returns_active = np.diff(np.log(active_data), axis=0)

In [15]:
n_rows = min(len(returns_active), len(factors), len(returns_SPX))
factors = factors[:n_rows]
returns_active = returns_active[:n_rows]
returns_SPX = returns_SPX[:n_rows]

In [ ]:
import statsmodels.api as sm
from statsmodels.api import OLS

y = returns_SPX
X1 = np.hstack([np.ones((n_rows, 1)), returns_active])

# Model 1
model1 = OLS(y, X1).fit()
R2_1 = model1.rsquared
adjusted_R2_1 = model1.rsquared_adj

# Model 2
X2 = np.hstack([np.ones((n_rows, 1)), returns_active, factors[:, 1:], factors[:, 2:]])
model2 = OLS(y, X2).fit()
R2_2 = model2.rsquared
adjusted_R2_2 = model2.rsquared_adj

print(model1.summary())

In [17]:
from sklearn.metrics import mean_squared_error

# In-sample and out-of-sample split
split_index = int(n_rows * 0.8)
X_in1, X_out1 = X1[:split_index], X1[split_index:]
y_in1, y_out1 = y[:split_index], y[split_index:]
X_in2, X_out2 = X2[:split_index], X2[split_index:]
y_in2, y_out2 = y[:split_index], y[split_index:]

# Out-of-sample predictions
model_in1 = OLS(y_in1, X_in1).fit()
y_fitted1 = model_in1.predict(X_out1)
MSE1 = mean_squared_error(y_out1, y_fitted1)

model_in2 = OLS(y_in2, X_in2).fit()
y_fitted2 = model_in2.predict(X_out2)
MSE2 = mean_squared_error(y_out2, y_fitted2)

In [18]:
# Random Forest model

from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(n_estimators=16, random_state=42)
rf_model.fit(X_in1, y_in1)
y_pred_rf = rf_model.predict(X_out1)
MSE_RF = mean_squared_error(y_out1, y_pred_rf)

In [ ]:
# Feedforward Neural Network model and Hyperparameter Tuning

from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

param_grid = {
    'hidden_layer_sizes': [(10,), (20,), (10, 10), (10, 8), (20, 10, 5)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate_init': [0.001, 0.01, 0.1],
    'max_iter': [500, 1000]
}

nn_model = MLPRegressor(random_state=77)

grid_search = GridSearchCV(estimator=nn_model, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=5, verbose=2, n_jobs=-1)

grid_search.fit(X_in2, y_in2)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

y_pred_nn = best_model.predict(X_out2)
MSE_NN = mean_squared_error(y_out2, y_pred_nn)

print("Best Parameters:", best_params)
print("MSE for the Best Neural Network Model:", MSE_NN)


In [20]:
# Diebold-Mariano test

from scipy.stats import ttest_rel

dm_stat_rf, p_value_rf = ttest_rel(y_out2, y_pred_rf)
dm_stat_nn, p_value_nn = ttest_rel(y_out2, y_pred_nn)

In [21]:
# Fluctuation test (Giacomini-Rossi)

from statsmodels.tsa.stattools import adfuller

def fluctuation_test_alternative(errors, critical_value=1.96):
    adf_stat, p_value, _, _, _, _ = adfuller(errors, autolag="AIC")
    fluctuation_detected = adf_stat > critical_value
    return adf_stat, p_value, fluctuation_detected

forecast_errors_nn = y_out2 - y_pred_nn
forecast_errors_rf = y_out1 - y_pred_rf
forecast_errors_model2 = y_out2 - y_fitted2

adf_nn_stat, adf_nn_p_value, nn_fluctuation = fluctuation_test_alternative(forecast_errors_nn)
adf_rf_stat, adf_rf_p_value, rf_fluctuation = fluctuation_test_alternative(forecast_errors_rf)
adf_model2_stat, adf_model2_p_value, model2_fluctuation = fluctuation_test_alternative(forecast_errors_model2)

fluctuation_alternative_results = {"ADF_Stat_NN": adf_nn_stat, "P_Value_NN": adf_nn_p_value, "Fluctuation_Detected_NN": nn_fluctuation, "ADF_Stat_RF": adf_rf_stat, "P_Value_RF": adf_rf_p_value, "Fluctuation_Detected_RF": rf_fluctuation, "ADF_Stat_Model2": adf_model2_stat, "P_Value_Model2": adf_model2_p_value, "Fluctuation_Detected_Model2": model2_fluctuation}

In [ ]:
# Figures

import matplotlib.pyplot as plt

# Figure 1: Out-of-Sample Predictions: Random Forest
plt.figure()
plt.plot(y_out1, 'k', linewidth=2, label='Actual')
plt.plot(y_pred_rf, ':r', linewidth=2, label=f'RF Prediction (MSE: {MSE_RF:.4e})')
plt.title('Out-of-Sample Predictions: Random Forest (Optimal)')
plt.xlabel('Observation')
plt.ylabel('Values')
plt.legend(loc='best')
plt.grid(True)
plt.show()

# Figure 2: Out-of-Sample Predictions: Model 1
plt.figure()
plt.plot(y_out1, 'k', linewidth=2, label='Actual')
plt.plot(y_fitted1, '--b', linewidth=2, label=f'Linear Prediction (Model 1, MSE: {MSE1:.4e})')
plt.title('Out-of-Sample Predictions: Model 1')
plt.xlabel('Observation')
plt.ylabel('Values')
plt.legend(loc='best')
plt.grid(True)
plt.show()

# Figure 3: Out-of-Sample Predictions: Model 2
plt.figure()
plt.plot(y_out2, 'k', linewidth=2, label='Actual')
plt.plot(y_fitted2, '--g', linewidth=2, label=f'Linear Prediction (Model 2, MSE: {MSE2:.4e})')
plt.title('Out-of-Sample Predictions: Model 2')
plt.xlabel('Observation')
plt.ylabel('Values')
plt.legend(loc='best')
plt.grid(True)
plt.show()
